In [18]:
from nltk import ne_chunk, pos_tag, word_tokenize
from nltk.tree import Tree
from nltk.chunk.regexp import RegexpParser
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from bs4 import BeautifulSoup
from collections import Counter, Iterable
from bs4.element import Comment
import requests, re, urllib, warnings, operator
from urllib.parse import urlparse
from urllib.error import HTTPError
from nltk.corpus import wordnet as wn
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

model = Doc2Vec.load("/Users/vivek/doc2vec/doc2vec.bin")

In [24]:
def checkAppropriateURL(url):
    parsedURL = urlparse(url)
    extn = parsedURL.path.split(".")[-1]
    site = parsedURL.netloc.split(".")
    path = parsedURL.path.split("/")[-1]
    if extn in ["pdf","doc","docx","xls","xlsx","ppt","pptx","odt"]:
        return False
    if any(i in site for i in ["pinterest"]):
        return False
    try:
        r = dict(requests.head(url).headers)["Content-Type"].split(";")[0]
        if(not r=="text/html"):
            print ("Failed",url,r)
            return False
    except:
        pass
    return True

def scrapeGoogleForAbstracts(query, count):
    global request
    words = query.split()
    q = ("+").join(words)
    print (q)
    query_url_goog = "https://www.google.co.in/search?q=\"" + q + "\""
    
    request += 1
    print ("Making request ", request)
    try:
        data = requests.get(query_url_goog).text
    except Exception as e:
        print (e)
        return
    if "detected unusual traffic" in data.lower():
        print ("Rate limited")
        return
    time.sleep(sleepingTime)
#     print (data)
    soup = BeautifulSoup(data, "lxml")
    h3Rows = [el for el in soup.find_all("div", {"class":"BNeawe s3v9rd AP7Wnd"}) if el.find("div", {"class":"BNeawe s3v9rd AP7Wnd"})]
    spanRows = [el for el in soup.find_all("div", {"class":"kCrYT"}) if el.find("a")]
    extra = len(spanRows) - len(h3Rows)
    spanRows2 = spanRows[extra:]
    zippedRows = zip(spanRows2,h3Rows)
    urls = []
#     print (list(zippedRows))
    for zippedRow in zippedRows:
        try:
            h3Row = zippedRow[0]
            url = h3Row.find("a")['href']
            if(url.split("?")[0]=="/search"):
                continue
        except (GeneratorExit, KeyboardInterrupt, SystemExit):
            raise
        except:
            continue
        par = urllib.parse.parse_qs(urlparse(url).query)
        try:
            appendingURL = par['q'][0]
        except:
            continue
        if(not checkAppropriateURL(appendingURL)):
            continue
        matchedQuery = query
        allbElems = zippedRow[1].findAll('b')
        for bElem in allbElems:
            if bElem.string[0].isalnum():
                matchedQuery = bElem.string
        urls.append((matchedQuery,appendingURL))
    
    i = 1
    same = 0
    prev = 0
    jj = 0
    exceptiontime = 0
    while(len(urls)<count):
#         print ("Query: ",query)
        tempurl = query_url_goog + "&start=" +str(i*10)
        i+=1

        request += 1
        print ("Making request ", request)
        try:
            data = requests.get(tempurl).text
            exceptiontime = 0
        except Exception as e:
            print (e)
            exceptiontime += 1
            if exceptiontime > 3:
                break
            continue
        time.sleep(sleepingTime)
        soup = BeautifulSoup(data, "lxml")
        h3Rows = [el for el in soup.find_all("div", {"class":"BNeawe s3v9rd AP7Wnd"}) if el.find("div", {"class":"BNeawe s3v9rd AP7Wnd"})]
        spanRows = [el for el in soup.find_all("div", {"class":"kCrYT"}) if el.find("a")]
        extra = len(spanRows) - len(h3Rows)
        spanRows2 = spanRows[extra:]
        zippedRows = zip(spanRows2, h3Rows)
        for (j,zippedRow) in enumerate(zippedRows):
            try:
                h3Row = zippedRow[0]
                url = h3Row.find("a")['href']
            except (GeneratorExit, KeyboardInterrupt, SystemExit):
                raise
            except:
                continue
            par = urllib.parse.parse_qs(urlparse(url).query)
            try:
                appendingURL = par['q'][0]
                if(not checkAppropriateURL(appendingURL)):
                    continue
                matchedQuery = query
                allbElems = zippedRow[1].findAll('b')
                for bElem in allbElems:
                    if bElem.string[0].isalnum():
                        matchedQuery = bElem.string
                urls.append((matchedQuery, appendingURL))
            except (GeneratorExit, KeyboardInterrupt, SystemExit):
                raise
            except:
                continue
        if prev == len(urls):
            same +=1 
            if same>=10:
                same = 0
                break
        prev = len(urls)
    return urls[:count]

def getCluesPatternsTuple(word):
    patterns = []
    patterns.append(("such as " + word, False))
    patterns.append((word + " and other", True))
    patterns.append((word + " or other", True))
    patterns.append(("including " + word, False))
    patterns.append(("especially " + word, False))
    return (patterns)


def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True


def text_from_html(body):
    # Scraping a Web Document

    soup = BeautifulSoup(body, 'html.parser')
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)  
    return u" ".join(t.strip() for t in visible_texts)

def getTextFromURL(url):
    hdr = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
   'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
   'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
   'Accept-Encoding': 'none',
   'Accept-Language': 'en-US,en;q=0.8',
   'Connection': 'keep-alive'}
    # print (url)
    html = urllib.request.urlopen(urllib.request.Request(url, headers=hdr))
    inputText = text_from_html(html)
    return inputText

def getAbstracts(query, count):
    urls = scrapeGoogleForAbstracts(query, 5)
    # print ("lenght",len(urls))
    texts = []
    for (q,url,title) in urls:
        print (url)
        try:
            text = getTextFromURL(url)
        except (GeneratorExit, KeyboardInterrupt, SystemExit):
            raise
        except:
            continue
        texts.append((q,url,text.strip(),title))
    return texts


In [19]:

def flatten(l):
    for el in l:
        if isinstance(el, Iterable) and not isinstance(el, (str, bytes)):
            yield from flatten(el)
        else:
            yield el

def getHypernymsFromWordnet(word):
    allSynsets = wn.synsets(word)
    multiList = [[[a.name() for a in b.lemmas()] for b in c.hypernyms()]for c in wn.synsets(word)]
    finalList = list(flatten(multiList))
    return finalList

def getInstances(text):
    grammar = """
        PRE:   {<NNS|NNP|NN|NP|JJ|UH>+}
        MID: {<DT|IN|POS|FW|-|NP|NPS|NN|NNS>+}
        INSTANCE:   {(<DT+>)?(<JJ+>)?<PRE>(<MID><PRE>)?}
    """
    chunker = RegexpParser(grammar)
    taggedText = pos_tag(word_tokenize(text))
    textChunks = chunker.parse(taggedText)
    current_chunk = []
    for i in textChunks:
        if (type(i) == Tree and i.label() == "INSTANCE"):
            # print (i.leaves())
            current_chunk.append(" ".join([token for token, pos in i.leaves()]))
    return current_chunk

def getConcepts(text):
    grammar = """
        CONCEPT:   {(<DT>)?(<JJ>)?<NN|NNS>+}
    """
    chunker = RegexpParser(grammar)
    taggedText = pos_tag(word_tokenize(text))
    textChunks = chunker.parse(taggedText)
    current_chunk = []
    for i in textChunks:
        if (type(i) == Tree and i.label() == "CONCEPT"):
            current_chunk.append(" ".join([token for token, pos in i.leaves()]))
    return current_chunk

stopwords_en = stopwords.words("english")

def preprocessing(raw):
    wordlist = word_tokenize(raw)
    text = [w.lower () for w in wordlist if w not in stopwords_en]
    return text

def simDoc(text1, text2):
    f1 = text1.split()
    f2 = text2.split()
    sim = model.docvecs.similarity_unseen_docs(model, f1, f2, steps=100)
    return sim


In [15]:
def extractConcepts(posList, text, query):
    allInstances = []
    for (st,end) in posList:
        if query[1]:
            #Instances after, and exists
            tempText = text[end+1:]
            rest = re.match(".*?\.",tempText)
            if rest:
                rest = rest.group()
            else:
                rest = tempText
            andExists = re.search(" (and|or) ", rest)
            if andExists:
                print ("rest:", rest)
                lastConceptPos = andExists.end()
                lastConcept = rest[lastConceptPos:]
                print ("lastConcept: ", lastConcept)
                lastConceptExtracted = getConcepts(lastConcept.strip())
                if lastConceptExtracted:
                    allInstances.append(lastConceptExtracted[0])
                prevConceptsEnd = andExists.start()
                prevConcepts = rest[:prevConceptsEnd]
                prevConceptsList = prevConcepts.split(",")
                prevConceptsList = [a for a in prevConceptsList if a.strip()]
                print ("****START")
                print ("previous Concepts:",prevConcepts)
                for concept in prevConceptsList:
                        print (concept.strip(), getConcepts(concept.strip()))
                        conceptExtracted = getConcepts(concept.strip())
                        if conceptExtracted:
                            allInstances.append(conceptExtracted[-1])
                print ("*****DONE")
            else:
                allConcepts = rest.split(",")
                print ("#####HERE ARE ALL CONCEPTS:",allConcepts)
                for (i,concept) in enumerate(allConcepts):
                        if i==(len(allConcepts)-1):
                            print (concept)
                            conceptExtracted = getConcepts(concept.strip())
                            print ("1######CONCEPT EXTRACTED:", conceptExtracted)
                            if conceptExtracted:
                                allInstances.append(conceptExtracted[0])
                        else:
                            conceptExtracted = getConcepts(concept.strip())
                            print ("######CONCEPT EXTRACTED:", conceptExtracted)
                            if conceptExtracted:
                                allInstances.append(conceptExtracted[-1])
        else:
            tempText = text[:st]
            print ("tempText", tempText)
            try:
                lastSentPos = [m.end() for m in re.finditer(".*?\.",tempText)][-1]
                lastSent = tempText[:lastSentPos]
            except:
                lastSent = tempText
            lastSentList = lastSent.split(",")
            lastSentList = [a for a in lastSentList if a.strip()]
            if len(lastSentList) == 1:
                conceptExtracted = getConcepts(lastSentList[0].strip())
                if conceptExtracted:
                    allInstances.append(conceptExtracted[-1])
            else:
                for (i,concept) in enumerate(lastSentList):
                    conceptExtracted = getConcepts(concept.strip())
                    if conceptExtracted:
                        if i==(len(lastSentList)-1):
                            allInstances.append(conceptExtracted[0])
                        else:
                            allInstances.append(conceptExtracted[-1])
    return allInstances

In [26]:
def getHypernymsFromWWW(instance):
    cluesPatternsTuple = getCluesPatternsTuple(instance)
    hits = {}
    for c,(clue,isAfter) in enumerate(cluesPatternsTuple):
         
        allAbstracts = getAbstracts(clue,10)
        # print (allAbstracts)
        for (query,url,abstract,title) in allAbstracts:
            filename = ''.join(e for e in url if e.isalnum())
            f = open(filename[:15]+".txt","w+")
            f.write(abstract)
            f.close()

        for cnt,(query,url,abstract,title) in enumerate(allAbstracts):
            print (len(abstract))
            text1 = preprocessing(abstract)
            text2 = preprocessing(text)
            print ("ssF",len(text1),len(text2))
            if(not len(text1)):
                continue
            similarity = simDoc(text1, text2)

            if similarity>threshold:
                print (c,"::",cnt,":",url)
                startEndPositions = []
                try:
                    for m in re.finditer(re.escape(query), abstract):
                        try:
                            startEndPositions.append((m.start(), m.end()))
                        except:
                            print ("query:",query,"url:",url)
                            titleSearch = re.search(clue, title)
                            if titleSearch:
                                startEndPositions.append((titleSearch.start(), titleSearch.end()))
                            else:
                                continue
                except:
                    print ("1","query:",q,"url:",url)
                    continue
                allConcepts = extractConcepts(startEndPositions, abstract, query)
                if(allConcepts):
                    for conc in allConcepts:
                        if conc in hits:
                            hits[conc] += 1
                        else:
                            hits[conc] = 1
                print ("Number,",c," All concepts:", allConcepts)
    
    print ("Done for instance", instance)
    concept = max(hits.items(), key=operator.itemgetter(1))[0]
    return (concept, hits)


In [27]:

def URLInput():
    url = 'https://www.webstaurantstore.com/article/101/types-of-pizza.html'
    text = getTextFromURL(url)
    return text

def DocInput():
    file = open("pizza.txt")
    text = file.read()
    return text



def main(text, threshold):
#     instances = getInstances(text)
    instances = [l.strip() for l in open("../../HypeNET/concepts_pizza_relevant2.txt","r").read().split("\n")]
    finalList = []
    for instance in instances:
        #Add Code for iterating through patterns and clues here
        
        concept = getHypernymsFromWWW(instance)
        finalList.append((concept,instance))    

        concept = getHypernymsFromWordnet(instance)
        finalList.append((concept,instance))
        print ("Doing instance",instance)
    print ("Here's the final list of instances: ",finalList)
    return finalList

# text = URLInput()
text = DocInput()
threshold = 0.3
main(text, threshold)

such+as+pizzas
par {'q': ['https://economictimes.indiatimes.com/industry/cons-products/food/biryani-now-rules-indias-fast-food-landscape-and-heres-how-it-helped-emerge-a-lot-of-start-ups/articleshow/65273024.cms'], 'sa': ['U'], 'ved': ['0ahUKEwikptzQ6NjfAhUNcCsKHbnEBzYQFggUMAA'], 'usg': ['AOvVaw2hLByHf029cMQSwMTtB0kX']}
par {'q': ['https://www.hindustantimes.com/fitness/do-you-associate-pizzas-with-parties-your-weight-may-determine-how-you-think-of-food/story-L81DfeHEmSLxe320Nl2qaM.html'], 'sa': ['U'], 'ved': ['0ahUKEwikptzQ6NjfAhUNcCsKHbnEBzYQFggaMAE'], 'usg': ['AOvVaw1vvnM56nryuX5TFkw0NHR1']}
par {'q': ['https://www.quora.com/Why-is-that-junk-foods-such-as-Pizzas-Burgers-Pastas-and-Maggie-are-allowed-to-be-sold-in-India'], 'sa': ['U'], 'ved': ['0ahUKEwikptzQ6NjfAhUNcCsKHbnEBzYQFgggMAI'], 'usg': ['AOvVaw1rZQC9eNhl4iUXwer0FrKO']}
par {'q': ['https://www.indeed.co.uk/Food-Management-jobs-in-Spalding'], 'sa': ['U'], 'ved': ['0ahUKEwikptzQ6NjfAhUNcCsKHbnEBzYQFggjMAM'], 'usg': ['AOvVaw1I4P

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:73: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:75: DeprecationWarning: Call to deprecated `n_similarity` (Method will be removed in 4.0.0, use self.wv.n_similarity() instead).
/usr/local/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


FF 494 1618
0 :: 1 : https://www.hindustantimes.com/fitness/do-you-associate-pizzas-with-parties-your-weight-may-determine-how-you-think-of-food/story-L81DfeHEmSLxe320Nl2qaM.html
rest: are associated with their function or the context in which they are eaten.
lastConcept:  the context in which they are eaten.
****START
previous Concepts: are associated with their function
are associated with their function ['function']
*****DONE
rest: are associated with parties or picnics.
lastConcept:  picnics.
****START
previous Concepts: are associated with parties
are associated with parties ['parties']
*****DONE
rest: are generally associated with their function or the context in which they are eaten.
lastConcept:  the context in which they are eaten.
****START
previous Concepts: are generally associated with their function
are generally associated with their function ['function']
*****DONE
rest: are generally associated with their function or the context in which they are eaten such as parties o

****START
previous Concepts: or other products,
or other products ['other products']
*****DONE
rest: or other products, and an operating method   Info   Publication number  WO2011045662A1    WO2011045662A1  PCT/IB2010/002623  IB2010002623W  WO2011045662A1  WO 2011045662 A1  WO2011045662 A1  WO 2011045662A1        IB 2010002623 W  IB2010002623 W  IB 2010002623W  WO 2011045662 A1  WO2011045662 A1  WO 2011045662A1  Authority  WO  Grant status  Application   Patent type     Prior art keywords  food product  heating  position  heating device  chamber  Prior art date  2009-10-16   Application number  PCT/IB2010/002623  Other languages    French ( fr )   Inventor  Berardo Poli  Silvio Rocchelli  Mauro Righi  Original Assignee  Cierreci S.
lastConcept:  an operating method   Info   Publication number  WO2011045662A1    WO2011045662A1  PCT/IB2010/002623  IB2010002623W  WO2011045662A1  WO 2011045662 A1  WO2011045662 A1  WO 2011045662A1        IB 2010002623 W  IB2010002623 W  IB 2010002623W  WO 2

****START
previous Concepts: or other products, in an automatic distributor apparatus,  the apparatus comprising:  at least a collection chamber (10) capable of containing a plurality of food products (11 ),  a second chamber (30),  means for heating the products (11 ), located in the second chamber (30), comprising a heating device (35), in a superior position, having an active heating surface (36) facing downwards,
or other products ['other products']
in an automatic distributor apparatus ['an automatic distributor apparatus']
the apparatus comprising:  at least a collection chamber (10) capable of containing a plurality of food products (11 ) ['the apparatus comprising', 'a collection chamber', 'capable', 'a plurality', 'food products']
a second chamber (30) ['a second chamber']
means for heating the products (11 ) ['means', 'the products']
located in the second chamber (30) ['the second chamber']
comprising a heating device (35) ['a heating device']
in a superior position ['a super

...

https://www.facebook.com/pages/Especially-Pizzas/371301062451
https://www.facebook.com/fifimanini/photos/allegra-loves-pizza-especially-pizzas-that-are-made-personally-by-her-papa-thehe/1843848759009637/
https://www.tripadvisor.in/ShowUserReviews-g635900-d1818127-r96174962-Ascot_Hotel-Buyukada_Princes_Islands.html
https://foursquare.com/top-places/bangalore/best-places-pizza
http://www.jubilantfoodworks.com/2012/04/the-nano-pizza/
http://www.businessworld.in/article/The-Nano-Pizza/08-11-2014-67998
https://giordanos.com/how-pizza-eating-style-reflects-personality/
9572
ssF 1217 1618
FF 1217 1618
4 :: 0 : https://www.tripadvisor.com.ph/ShowUserReviews-g189932-d782718-r381626971-Restaurant_KG-Espoo_Uusimaa.html
#####HERE ARE ALL CONCEPTS: ['- Restaurant KG     Europe   Finland   Uusimaa   Espoo   Espoo Restaurants   Restaurant KG                 “Good food', ' especially pizzas ”      Review of Restaurant KG                                                          20 photos       Restaura

par {'q': ['https://www.refrigeratedfrozenfood.com/articles/83570-pizza-processing-spotlight'], 'sa': ['U'], 'ved': ['0ahUKEwjvtvi76djfAhVMcCsKHWmlDeYQFgg2MAg'], 'usg': ['AOvVaw0vrG_yf7QuK-QDPZ4ptT3Y']}
par {'q': ['https://transitiondarebin.org/2015/03/17/a-message-from-our-friends-at-the-pizza-club/'], 'sa': ['U'], 'ved': ['0ahUKEwjvtvi76djfAhVMcCsKHWmlDeYQFgg8MAk'], 'usg': ['AOvVaw1LOMIYYYrEB8q6KnpgjNtN']}
Failed http://www.bakingbiscuit.com/in-depth-information/chilled-is-the-new-freshness.html?file=files/f2m-media/pdf/archiv/baking%20and%20biscuit/issue%202012-03/2012-03_18_Chilled_is_the_new_freshness.pdf application/pdf
http://www.hellthyjunkfood.com/pizza-grilled-cheese/
http://www.hellthyjunkfood.com/pizza-grilled-cheese/
https://www.youtube.com/watch?v=MsF6daq4_tk
https://farrp.unl.edu/informallfish
https://www.refrigeratedfrozenfood.com/articles/83570-pizza-processing-spotlight
https://transitiondarebin.org/2015/03/17/a-message-from-our-friends-at-the-pizza-club/
"such as piz

https://context.reverso.net/translation/english-spanish/toppings+on+pizza
https://www.linguee.com/german-english/translation/pizzabelag.html
http://digital.bnpmedia.com/article/PIZZA/2809033/414422/article.html
http://www.cheesemancorp.com/BurkeCorp.html
https://www.bizjournals.com/sanfrancisco/stories/1996/09/30/newscolumn2.html
http://twobirdskitchen.co.uk/menus-and-price-list/
http://redtri.com/san-francisco-events/bakers-alley-at-the-school-farm
http://www.housewares.org/housewaresconnect365/detail?com_uid=00030332
4687
ssF 653 1618
FF 653 1618
2 :: 0 : https://www.tes.com/teaching-resource/count-1-to-10-how-many-pizza-toppings-counting-activity-11759755
#####HERE ARE ALL CONCEPTS: ['manipulatives to place on the pizza slice to count.']
manipulatives to place on the pizza slice to count.
1######CONCEPT EXTRACTED: ['manipulatives', 'the pizza slice', 'count']
Number, 2  All concepts: ['manipulatives']
3517
ssF 501 1618
FF 501 1618
2 :: 1 : https://www.teacherspayteachers.com/Product

FF 1275 1618
3 :: 7 : https://markets.businessinsider.com/news/stocks/hormel-foods-adds-to-its-foodservice-portfolio-with-the-acquisition-of-fontanini-italian-meats-and-sausages-1002265438
#####HERE ARE ALL CONCEPTS: ['and meatballs.']
and meatballs.
1######CONCEPT EXTRACTED: ['meatballs']
Number, 3  All concepts: ['meatballs']
especially+pizza+toppings
par {'q': ['https://www.ukessays.com/essays/business/strategic-analysis-and-recommendations-for-dominos-pizza-business-essay.php'], 'sa': ['U'], 'ved': ['0ahUKEwjtgI6Y6tjfAhUPbisKHaaJCswQFgggMAU'], 'usg': ['AOvVaw0vU0bM9Uf7f9H6LAswpvzw']}
par {'q': ['https://chuckrybak.com/uw-update-open-letter-to-speaker-robin-vos/'], 'sa': ['U'], 'ved': ['0ahUKEwjtgI6Y6tjfAhUPbisKHaaJCswQFggnMAY'], 'usg': ['AOvVaw0_r4WfdEPQcWyuqAZx15gx']}
par {'q': ['https://nutritionalnatasa.com/2018/06/30/pizza-hut-vegan-options-vegan-cheese/'], 'sa': ['U'], 'ved': ['0ahUKEwjtgI6Y6tjfAhUPbisKHaaJCswQFggtMAc'], 'usg': ['AOvVaw0X8-RZR_1raPK_2KrSK_r0']}
par {'q': ['htt

['pizzas', 'pizza toppings']